In [1]:
import pandas as pd
import numpy as np
import json
import importlib, sys, os, requests
from datetime import date, datetime
from bs4 import BeautifulSoup

from pathlib import Path
sys.path.append(str(Path(os.path.abspath('')).parents[0]))

import seaborn as sns
import matplotlib.pyplot as plt
import support_library as sl

_ = [importlib.reload(sys.modules[m]) for m in list(sys.modules) if 'support_library' in m]

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
# Leitura do Excel
df = pd.read_excel('__knime_producao_anual.xlsx')
df['Market_Year'] = df['Market_Year'].astype(int)
# df.set_index(['Market_Year', 'Commodity_Description'], drop=False, inplace=True)
print(df.shape)
df.head()

(198, 6)


E:\Projetos\ITA_Tese\.env-win-tese\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value
0,Coconut,World,2000,Production,(1000 MT),3498.000
1,Coconut,World,2001,Production,(1000 MT),3074.000
2,Coconut,World,2002,Production,(1000 MT),3058.000
3,Coconut,World,2003,Production,(1000 MT),3161.000
4,Coconut,World,2004,Production,(1000 MT),3316.000


In [3]:
df_bk = df.copy()

In [4]:
df= df_bk.copy()

In [5]:
# Obtido a variação do periodo YoY
df['yoy'] = df.sort_values(['Commodity_Description', 'Reference', 'Attribute_Description', 'Market_Year']).groupby(['Commodity_Description','Reference','Attribute_Description'])['Value'].pct_change()
df['yoy'].fillna(0, inplace=True)
print(df.shape)
df.head()

(198, 7)


,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value,yoy
0,Coconut,World,2000,Production,(1000 MT),3498.000,0.000
1,Coconut,World,2001,Production,(1000 MT),3074.000,-0.121
2,Coconut,World,2002,Production,(1000 MT),3058.000,-0.005
3,Coconut,World,2003,Production,(1000 MT),3161.000,0.034
4,Coconut,World,2004,Production,(1000 MT),3316.000,0.049


In [17]:
# Configurando o campo data para ser o ultimo dia no ano
df['date'] = df['Market_Year'].astype(str) + f'-1-1'
df['date'] = pd.to_datetime(df['date'] , format='%Y-%m-%d')
df['date'] = df['date'] + pd.DateOffset(years=1) + pd.Timedelta(days=-1) 
df.set_index(['date'], inplace = True)

# df.index.rename('date_idx', inplace=True)
# df.head()
df.head()

,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value,yoy
date,,,,,,,
2000-12-31,Coconut,World,2000,Production,(1000 MT),3498.000,0.000
2001-12-31,Coconut,World,2001,Production,(1000 MT),3074.000,-0.121
2002-12-31,Coconut,World,2002,Production,(1000 MT),3058.000,-0.005
2003-12-31,Coconut,World,2003,Production,(1000 MT),3161.000,0.034
2004-12-31,Coconut,World,2004,Production,(1000 MT),3316.000,0.049


In [18]:
df['Commodity_Description'].unique()

array(['Coconut', 'Cottonseed', 'Olive', 'Palm', 'Palm Kernel', 'Peanut',
       'Rapeseed', 'Soybean', 'Sunflower'], dtype=object)

In [19]:
# filtrando somente sunflower
sun = df[df['Commodity_Description'] == 'Sunflower']
sun['year_value'] = sun['Value']
sun

C:\Users\fkfouri\AppData\Local\Temp\ipykernel_12392\748614064.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sun['year_value'] = sun['Value']


,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value,yoy,year_value
date,,,,,,,,
2000-12-31,Sunflower,World,2000,Production,(1000 MT),7999.000,0.000,7999.000
2001-12-31,Sunflower,World,2001,Production,(1000 MT),7339.000,-0.083,7339.000
2002-12-31,Sunflower,World,2002,Production,(1000 MT),8011.000,0.092,8011.000
2003-12-31,Sunflower,World,2003,Production,(1000 MT),9107.000,0.137,9107.000
2004-12-31,Sunflower,World,2004,Production,(1000 MT),9185.000,0.009,9185.000
2005-12-31,Sunflower,World,2005,Production,(1000 MT),10562.000,0.150,10562.000
2006-12-31,Sunflower,World,2006,Production,(1000 MT),10742.000,0.017,10742.000
2007-12-31,Sunflower,World,2007,Production,(1000 MT),10257.000,-0.045,10257.000
2008-12-31,Sunflower,World,2008,Production,(1000 MT),11891.000,0.159,11891.000


In [20]:
#Realizando o resample por mes
sun2 = sun.groupby(['Commodity_Description','Reference','Attribute_Description'])[['Value', 'yoy', 'year_value']].resample('M').mean().reset_index()
sun2.set_index('date', inplace=True)
sun_bk = sun2.copy()
sun_bk

,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value
date,,,,,,
2000-12-31,Sunflower,World,Production,7999.000,0.000,7999.000
2001-01-31,Sunflower,World,Production,NaN,NaN,NaN
2001-02-28,Sunflower,World,Production,NaN,NaN,NaN
2001-03-31,Sunflower,World,Production,NaN,NaN,NaN
2001-04-30,Sunflower,World,Production,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-08-31,Sunflower,World,Production,NaN,NaN,NaN
2021-09-30,Sunflower,World,Production,NaN,NaN,NaN
2021-10-31,Sunflower,World,Production,NaN,NaN,NaN


In [21]:
#Comparando diversos metodos de interpolaçao

sun2 = sun_bk.copy()
# sun2.set_index('date', inplace=True)
comp = sun2.groupby(pd.Grouper(freq='Y'))[['year_value']].sum()
methods = [ 'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial', 'krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline']
choice = 'linear'


for me in methods:
    temp = sun_bk.copy()
#     temp.set_index('date', inplace=True)
    temp[me] = (temp['Value']/12).interpolate(method=me, order=2)
    
    
    comp2 = temp.groupby(pd.Grouper(freq='Y'))[[me, 'year_value']].sum()
    comp2[f'err_{me}'] = comp2[me] / comp['year_value']
    
    if choice == me:
        sun2['value_interpolate'] = (sun2['Value']/12).interpolate(method=me, order=2)
        sun2['err_1']  = comp2[me] / comp['year_value']
        sun2['err_2']  = comp['year_value'] - comp2[me]
#         sun2['err'] = sun2['err'].interpolate(method='pad', limit_direction='forward', order=2)
    

    comp2[f'err_{me}'] = comp['year_value'] - comp2[me]
    
    comp = pd.merge(comp, comp2[f'err_{me}'], left_index=True, right_index=True)

comp.reset_index()#.set_index('date')
comp.head(8)

E:\Projetos\ITA_Tese\.env-win-tese\lib\site-packages\scipy\interpolate\_polyint.py:324: RuntimeWarning: overflow encountered in multiply
  pi = w*pi
E:\Projetos\ITA_Tese\.env-win-tese\lib\site-packages\scipy\interpolate\_polyint.py:325: RuntimeWarning: invalid value encountered in add
  p += pi[:,np.newaxis] * self.c[k]
E:\Projetos\ITA_Tese\.env-win-tese\lib\site-packages\scipy\interpolate\_polyint.py:325: RuntimeWarning: invalid value encountered in multiply
  p += pi[:,np.newaxis] * self.c[k]


,year_value,err_linear,err_nearest,err_zero,err_slinear,err_quadratic,err_cubic,err_spline_x,err_barycentric,err_polynomial,err_krogh,err_piecewise_polynomial,err_spline_y,err_pchip,err_akima,err_cubicspline
date,,,,,,,,,,,,,,,,
2000-12-31,7999.000,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417,7332.417
2001-12-31,7339.000,-302.500,-330.000,-605.000,-303.932,-185.613,-177.579,-184.698,-3872662.004,-185.613,6727.417,-303.932,-184.698,-194.333,-165.462,-177.579
2002-12-31,8011.000,308.000,336.000,616.000,309.458,394.936,403.324,393.119,280056.311,394.936,7343.417,309.458,393.119,378.449,355.923,403.324
2003-12-31,9107.000,502.333,548.000,1004.667,504.711,454.645,438.535,456.830,-37195.277,454.645,8348.083,504.711,456.830,448.167,499.296,438.535
2004-12-31,9185.000,35.750,39.000,71.500,35.839,55.848,60.186,55.432,7752.043,55.848,8419.583,35.839,55.432,35.985,21.500,60.186
2005-12-31,10562.000,631.125,688.500,1262.250,634.112,656.924,672.841,654.186,-1447.573,656.924,9681.833,634.112,654.186,648.830,627.075,672.841
2006-12-31,10742.000,82.500,90.000,165.000,82.890,-38.935,-70.556,-34.333,674.913,-38.935,9846.833,82.890,-34.333,56.586,29.486,-70.556
2007-12-31,10257.000,-222.292,-242.500,-444.583,-223.344,-131.135,-110.118,-133.867,-460.758,-131.135,9402.250,-223.344,-133.867,-223.589,-202.398,-110.118


In [22]:
sun2.head(25)

,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value,value_interpolate,err_1,err_2
date,,,,,,,,,
2000-12-31,Sunflower,World,Production,7999.000,0.000,7999.000,666.583,0.083,7332.417
2001-01-31,Sunflower,World,Production,NaN,NaN,NaN,662.000,NaN,NaN
2001-02-28,Sunflower,World,Production,NaN,NaN,NaN,657.417,NaN,NaN
2001-03-31,Sunflower,World,Production,NaN,NaN,NaN,652.833,NaN,NaN
2001-04-30,Sunflower,World,Production,NaN,NaN,NaN,648.250,NaN,NaN
2001-05-31,Sunflower,World,Production,NaN,NaN,NaN,643.667,NaN,NaN
2001-06-30,Sunflower,World,Production,NaN,NaN,NaN,639.083,NaN,NaN
2001-07-31,Sunflower,World,Production,NaN,NaN,NaN,634.500,NaN,NaN
2001-08-31,Sunflower,World,Production,NaN,NaN,NaN,629.917,NaN,NaN


In [167]:
year = '2012'
month = 3
start = 0 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2-1
end = 1 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2

print(start,end,sun2.loc[year].shape[0])
sun2.loc[year][start:end]

5 6 12


,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value,value_interpolate,err
date,,,,,,,,
2012-06-30,Sunflower,World,Production,NaN,NaN,NaN,1134.708,0.927


In [168]:
# sun2['value_interpolate'] = sun2['value_interpolate'] + sun2['err']

In [169]:
sun2.loc[year]

,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value,value_interpolate,err
date,,,,,,,,
2012-01-31,Sunflower,World,Production,NaN,NaN,NaN,1186.479,0.927
2012-02-29,Sunflower,World,Production,NaN,NaN,NaN,1176.125,0.927
2012-03-31,Sunflower,World,Production,NaN,NaN,NaN,1165.771,0.927
2012-04-30,Sunflower,World,Production,NaN,NaN,NaN,1155.417,0.927
2012-05-31,Sunflower,World,Production,NaN,NaN,NaN,1145.062,0.927
2012-06-30,Sunflower,World,Production,NaN,NaN,NaN,1134.708,0.927
2012-07-31,Sunflower,World,Production,NaN,NaN,NaN,1124.354,0.927
2012-08-31,Sunflower,World,Production,NaN,NaN,NaN,1114.000,0.927
2012-09-30,Sunflower,World,Production,NaN,NaN,NaN,1103.646,0.927


In [170]:
# TEntativa para correcao em alguns meses


for idx, row in comp.iterrows():

    year = str(idx.year)
    
    months = [3, 9]
    
    for month in months:
        _idx = f'{year}-{month}'
        
        if _idx not in sun2.index:
            _idx = f'{year}-12'

#         start = 0 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2-1
#         end = 1 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2
#         value_interpolate = sun2.loc[year][start:end]['value_interpolate'][0]

        value_interpolate = sun2.loc[_idx]['value_interpolate'][0]
        correcao =  value_interpolate + row['err_linear']/len(months)
        index = sun2.loc[_idx].index[0]
    
#         print(_idx, row['err_linear'], start, end, "===>", value_interpolate, correcao)

        sun2.at[_idx, 'value_interpolate_meses'] = correcao
    
sun2.head(25)

,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value,value_interpolate,err,value_interpolate_meses
date,,,,,,,,,
2000-12-31,Sunflower,World,Production,7999.000,0.000,7999.000,666.583,0.083,4332.792
2001-01-31,Sunflower,World,Production,NaN,NaN,NaN,662.000,0.083,NaN
2001-02-28,Sunflower,World,Production,NaN,NaN,NaN,657.417,0.083,NaN
2001-03-31,Sunflower,World,Production,NaN,NaN,NaN,652.833,0.083,501.583
2001-04-30,Sunflower,World,Production,NaN,NaN,NaN,648.250,0.083,NaN
2001-05-31,Sunflower,World,Production,NaN,NaN,NaN,643.667,0.083,NaN
2001-06-30,Sunflower,World,Production,NaN,NaN,NaN,639.083,0.083,NaN
2001-07-31,Sunflower,World,Production,NaN,NaN,NaN,634.500,0.083,NaN
2001-08-31,Sunflower,World,Production,NaN,NaN,NaN,629.917,0.083,NaN


In [171]:
#Correcao do ponto central
for idx, row in comp.iterrows():

    year = str(idx.year)
    
    
    start = 0 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2-1
    end = 1 if sun2.loc[year].shape[0] == 1 else sun2.loc[year].shape[0]//2
    
    value_interpolate = sun2.loc[year][start:end]['value_interpolate'][0]
    correcao =  value_interpolate + row['err_linear']
    index = sun2.loc[year][start:end].index[0]
    
    print(year, row['err_linear'], start, end, "===>", value_interpolate, correcao)
#     sun2.at()9

    sun2.at[index, 'value_interpolate'] = correcao
    
sun2.head(25)

2000 7332.416666666667 0 1 ===> 666.5833333333334 7999.0
2001 -302.5 5 6 ===> 639.0833333333334 336.58333333333337
2002 308.0 5 6 ===> 639.5833333333334 947.5833333333334
2003 502.33333333333394 5 6 ===> 713.25 1215.583333333334
2004 35.75 5 6 ===> 762.1666666666666 797.9166666666666
2005 631.125 5 6 ===> 822.7916666666666 1453.9166666666665
2006 82.5 5 6 ===> 887.6666666666666 970.1666666666666
2007 -222.29166666666606 5 6 ===> 874.9583333333333 652.6666666666672
2008 748.9166666666679 5 6 ===> 922.8333333333333 1671.7500000000011
2009 105.875 5 6 ===> 1000.5416666666666 1106.4166666666665
2010 -16.95833333333394 5 6 ===> 1008.625 991.6666666666661
2011 1043.625 5 6 ===> 1101.9583333333333 2145.583333333333
2012 -683.375 5 6 ===> 1134.7083333333333 451.33333333333326
2013 1272.333333333334 5 6 ===> 1188.25 2460.583333333334
2014 -308.45833333333394 5 6 ===> 1275.875 967.4166666666661
2015 226.875 5 6 ===> 1268.4583333333333 1495.3333333333333
2016 1297.5416666666679 5 6 ===> 1407.0416

,Commodity_Description,Reference,Attribute_Description,Value,yoy,year_value,value_interpolate,err,value_interpolate_meses
date,,,,,,,,,
2000-12-31,Sunflower,World,Production,7999.000,0.000,7999.000,7999.000,0.083,4332.792
2001-01-31,Sunflower,World,Production,NaN,NaN,NaN,662.000,0.083,NaN
2001-02-28,Sunflower,World,Production,NaN,NaN,NaN,657.417,0.083,NaN
2001-03-31,Sunflower,World,Production,NaN,NaN,NaN,652.833,0.083,501.583
2001-04-30,Sunflower,World,Production,NaN,NaN,NaN,648.250,0.083,NaN
2001-05-31,Sunflower,World,Production,NaN,NaN,NaN,643.667,0.083,NaN
2001-06-30,Sunflower,World,Production,NaN,NaN,NaN,336.583,0.083,NaN
2001-07-31,Sunflower,World,Production,NaN,NaN,NaN,634.500,0.083,NaN
2001-08-31,Sunflower,World,Production,NaN,NaN,NaN,629.917,0.083,NaN


In [172]:
comp2 = sun2.groupby(pd.Grouper(freq='Y'))[['value_interpolate']].sum()
comp2

,value_interpolate
date,
2000-12-31,7999.000
2001-12-31,7339.000
2002-12-31,8011.000
2003-12-31,9107.000
2004-12-31,9185.000
2005-12-31,10562.000
2006-12-31,10742.000
2007-12-31,10257.000
2008-12-31,11891.000


In [174]:
#Para ser comparado com a grouper anterior
sun

,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value,yoy,year_value
date,,,,,,,,
2000-12-31,Sunflower,World,2000,Production,(1000 MT),7999.000,0.000,7999.000
2001-12-31,Sunflower,World,2001,Production,(1000 MT),7339.000,-0.083,7339.000
2002-12-31,Sunflower,World,2002,Production,(1000 MT),8011.000,0.092,8011.000
2003-12-31,Sunflower,World,2003,Production,(1000 MT),9107.000,0.137,9107.000
2004-12-31,Sunflower,World,2004,Production,(1000 MT),9185.000,0.009,9185.000
2005-12-31,Sunflower,World,2005,Production,(1000 MT),10562.000,0.150,10562.000
2006-12-31,Sunflower,World,2006,Production,(1000 MT),10742.000,0.017,10742.000
2007-12-31,Sunflower,World,2007,Production,(1000 MT),10257.000,-0.045,10257.000
2008-12-31,Sunflower,World,2008,Production,(1000 MT),11891.000,0.159,11891.000


In [100]:
#Upsampling

upsampled  = df.resample('M').mean()
interpolated = upsampled.interpolate(method='linear')
# print(upsampled.interpolate(method='linear') )
upsampled.head(25)

,Market_Year,Value,yoy
date,,,
2000-01-31,2000.0,10919.000000,0.000000
2000-02-29,NaN,NaN,NaN
2000-03-31,NaN,NaN,NaN
2000-04-30,NaN,NaN,NaN
2000-05-31,NaN,NaN,NaN
2000-06-30,NaN,NaN,NaN
2000-07-31,NaN,NaN,NaN
2000-08-31,NaN,NaN,NaN
2000-09-30,NaN,NaN,NaN


In [4]:
# # Obtido a variação do periodo YoY
# df['dd'] =df['Value'].rolling(3).mean()
# df.tail(50)

In [73]:
df= df_bk.copy()
rows = []
for idx, row in df.iterrows():
    for i in range(1, 12 + 1):
        temp = row.copy()
        temp['date'] = str(temp['Market_Year']) + f'-{i}-1'
        temp['date'] = pd.to_datetime(temp['date'] , format='%Y-%m-%d')
        if i < 12:
            temp.Value = np.NAN
        rows.append(temp)

df2 = pd.DataFrame( data=rows)
print(df2.shape)
df2.head(24)

(2376, 7)


,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value,date
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-01-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-02-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-03-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-04-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-05-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-06-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-07-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-08-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-09-01
0,Coconut,World,2000,Production,(1000 MT),NaN,2000-10-01


In [106]:
df2.expanding(12).mean().head(48)

C:\Users\fkfouri\AppData\Local\Temp\ipykernel_18616\3947665963.py:1: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Attribute_Description', 'Commodity_Description', 'Reference',
       'Unit_Description', 'date'],
      dtype='object')
  df2.expanding(12).mean().head(48)


,Market_Year,Value,yoy
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN
0,NaN,NaN,NaN


In [102]:
df2.expanding(1).std().head(48)

C:\Users\fkfouri\AppData\Local\Temp\ipykernel_18616\1058486018.py:1: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Attribute_Description', 'Commodity_Description', 'Reference',
       'Unit_Description', 'date'],
      dtype='object')
  df2.expanding(1).std().head(48)


,Market_Year,Value,yoy
0,NaN,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN
0,0.000000,NaN,NaN


In [17]:
list(range(1, 12 + 1))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [10]:
print(df.shape)
df.head()

(220, 7)


,Commodity_Code,Commodity_Description,Reference,Market_Year,Attribute_Description,Unit_Description,Value
0,4232000.0,Soybean,World,2000,Production,(1000 MT),26818.0
1,4232000.0,Soybean,World,2001,Production,(1000 MT),28948.0
2,4232000.0,Soybean,World,2002,Production,(1000 MT),30600.0
3,4232000.0,Soybean,World,2003,Production,(1000 MT),30282.0
4,4232000.0,Soybean,World,2004,Production,(1000 MT),32658.0


In [135]:
x = sun.groupby(['Commodity_Description','Reference','Attribute_Description','date'])[['Value', 'yoy']].mean().reset_index()
# x[x['Value'] == 1]
x

,Commodity_Description,Reference,Attribute_Description,date,Value,yoy
0,Sunflower,World,Production,2000-12-31,7999.0,0.000000
1,Sunflower,World,Production,2001-12-31,7339.0,-0.082510
2,Sunflower,World,Production,2002-12-31,8011.0,0.091566
3,Sunflower,World,Production,2003-12-31,9107.0,0.136812
4,Sunflower,World,Production,2004-12-31,9185.0,0.008565
5,Sunflower,World,Production,2005-12-31,10562.0,0.149918
6,Sunflower,World,Production,2006-12-31,10742.0,0.017042
7,Sunflower,World,Production,2007-12-31,10257.0,-0.045150
8,Sunflower,World,Production,2008-12-31,11891.0,0.159306
9,Sunflower,World,Production,2009-12-31,12122.0,0.019426
